In [5]:
import math
import os
import pandas as pd
import fitz
import pandas as pd
from todoist_api_python.api import TodoistAPI
from quarter_lib.akeyless import get_secrets
TODOIST_TOKEN = get_secrets("todoist/token")
TODOIST_API = TodoistAPI(TODOIST_TOKEN)


In [2]:
def filter_content(task):
    splitted = task['content'].split("\n\n")
    del (splitted[2])
    temp_book = splitted.pop(2)[4:]
    end = temp_book.find("'")
    task['book_title'] = temp_book[:end]
    rest = temp_book[end:][6:]
    end = rest.find("'")
    task['book_author'] = rest[:end]
    for lang_task, language in zip(splitted, ["de", "en"]):
        start = lang_task.find('"')
        end = lang_task.rfind('"')
        task[language + "_content"] = lang_task[start + 1:end]
    return task


def get_st_dev(list):
    mean = sum(list) / len(list)
    var = sum((l - mean) ** 2 for l in list) / len(list)
    return math.sqrt(var)

In [3]:
tasks = TODOIST_API.get_tasks(project_id="2281154095")
tasks = [filter_content(task.__dict__) for task in tasks]

In [7]:
test = pd.DataFrame(tasks)

In [10]:
test.groupby("book_title").groups

{'Klinische Psychologie und Psychotherapie, Band I': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], 'Models Attract Women Through Honesty': [12]}

In [23]:
doc = fitz.open("input.pdf")
for task in tasks[0:2]:
    language = "de"
    text_to_search = task[language + "_content"]
    print("text to search: " + text_to_search)
    splitted_search_text = text_to_search.split(" ")
    findings = []
    number_of_words_to_search = 4
    for index in range(len(splitted_search_text)):
        search_list = splitted_search_text[index: index + number_of_words_to_search]
        if (len(search_list) == number_of_words_to_search):
            words_to_search = (" ").join(search_list)
            for page in doc:
                text_instances = page.search_for(words_to_search)
                if len(text_instances):
                    findings.append((page, text_instances, text_to_search))
    print(f"found {len(findings)} entries for {text_to_search}")
    if (len(findings)):
        page_numbers = [page[0].number for page in findings]
        if (get_st_dev(page_numbers) < 1):
            try:
                x0y0 = findings[0][1][0][0]
                x0y1 = findings[0][1][0][1]
                x1y0 = findings[len(findings) - 1][1][0][2]
                x1y1 = findings[len(findings) - 1][1][0][3]
                inst = fitz.Rect(x0y0, x0y1, x1y0, x1y1)
                highlight = findings[0][0].add_highlight_annot(inst)
                text = findings[0][2]
                print("text: " + text)
                highlight.set_info({"content": text})
                highlight.update()
            except ValueError as e:
                print(e)
doc.save("output.pdf")

text to search: einige Autoren von erblichkeitseinflüssen von bis zu 60% für das Vorhandensein von Persönlichkeitsstörungen
found 6 entries for einige Autoren von erblichkeitseinflüssen von bis zu 60% für das Vorhandensein von Persönlichkeitsstörungen
text: einige Autoren von erblichkeitseinflüssen von bis zu 60% für das Vorhandensein von Persönlichkeitsstörungen
text to search: zahlungsebene der sogenannten Spielebene gelöst werden kann immer werden Verhaltensweisen eingesetzt
found 0 entries for zahlungsebene der sogenannten Spielebene gelöst werden kann immer werden Verhaltensweisen eingesetzt


In [107]:
findings[0][1][0][0]

178.18069458007812

In [86]:
list = [page[0].number for page in findings]

In [96]:
red = (1, 0, 0)
blue = (0, 0, 1)
gold = (1, 1, 0)
green = (0, 1, 0)


In [98]:
doc = fitz.open("input.pdf")

for page, instance in findings:
    highlight = page.add_highlight_annot(instance)
    highlight.set_oc(stroke=blue, fill=green)
    highlight.update()

doc.save("output.pdf", garbage=4, deflate=True, clean=True)

In [93]:
print(highlight)

'Highlight' annotation on page 174 of input.pdf


In [89]:
list

[174, 174, 174, 224, 174, 174]

In [87]:
list[3] = list[3] + 50

Standard deviation of the given list: 18.633899812498246


In [93]:
def highlight(doc, text, single_word_annotation_list):
    for page in doc:
        text_instances = page.search_for(text, quads=True)
        if (len(text_instances) > 0):
            if (text not in single_word_annotation_list):
                highlight = page.add_highlight_annot(text_instances)
                highlight.update()
            if (text.count(' ') == 0):
                single_word_annotation_list.append(text)

In [96]:
doc = fitz.open("B.pdf")
single_word_annotation_list = []

for line in lines:
    if (line.startswith("#")):
        continue
    elif (line.startswith("####")):
        continue
    elif (line.startswith("#####")):
        continue
    elif (line.startswith(">")):
        continue
    else:
        highlight(doc, line.strip(), single_word_annotation_list)

doc.save("output.pdf", garbage=4, deflate=True, clean=True)

single word annotation: phenomenology
single word annotation: phenomenology
single word annotation: phenomenology
single word annotation: phenomenology
single word annotation: phenomenology
single word annotation: phenomenology
single word annotation: phenomenology
single word annotation: phenomenology


In [98]:
from datetime import datetime
from dateutil import parser

In [102]:
test = datetime.now()

In [103]:
test1 = parser.parse("02/09/2022, 10:46:36")

In [105]:
(test - test1).seconds

208